In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import nfl_data_py as nfl

# 2024 Fantasy Football Analysis!

This year we are starting off with some lessons learned from 2023.

I will be focusing specifically this year on offensive positions.

Things we want to implement this year:
1. A UI to make the picking process more seamless
2. Dynamic picking process that matches the positions available each week.

Attributing Points:
- Touchdown = 7*(yds for play/net yards)
- Field Goal = 3*(yds for play/net yards)
- First Down = 7/10*(yds for play/net yards) = 0.7*(yds for play/net yards)

Things not considering:
1. laterals

### EDA!

Columns we Will Entertain:

1. ID Columns
    - game_id
    - play_id
    - home_team
    - away_team
    - season_type
    - posteam
    - defteam
    - touchdown
    - play_type
    - passer_player_id
    - passer_player_name
    - receiver_player_id
    - receiver_player_name
    - rusher_player_id
    - rusher_player_name
    - season
    - div_game
    - home_coach
    - away_coach
    

2. Analysis Columns
    - yards_gained
    - yds_net
    - incomplete_pass
    - interception
    - sack
    - series_result
    - sack_player_id
    - sack_player_name
    - fumbled_1_player_id
    - fumbled_1_player_name
    

In [3]:
pbp_data = (nfl.import_pbp_data([2018,2019,2020,2021,2022,2023], downcast=True, cache=False)
                                            [['game_id',
                                            'play_id',
                                            'home_team',
                                            'away_team',
                                            'season_type',
                                            'posteam',
                                            'defteam',
                                            'touchdown',
                                            'play_type',
                                            'passer_player_id',
                                            'passer_player_name',
                                            'receiver_player_id',
                                            'receiver_player_name',
                                            'rusher_player_id',
                                            'rusher_player_name',
                                            'season',
                                            'home_coach',
                                            'away_coach',
                                            'yards_gained',
                                            'ydsnet',
                                            'incomplete_pass',
                                            'interception',
                                            'sack',
                                            'series_result',
                                            'sack_player_id',
                                            'sack_player_name',
                                            'fumbled_1_player_id',
                                            'fumbled_1_player_name']])

points_attrib = {'First Down': 0.7,
                 'Touchdown': 7,
                 'Field Goal': 0.3}

pbp_data.loc[:, 'series_points'] = pbp_data.series_result.map(points_attrib)
pbp_data.loc[pbp_data.ydsnet != 0, 'net_points_gained'] = pbp_data.series_points * (pbp_data.yards_gained/pbp_data.ydsnet)
pbp_data.loc[pbp_data.net_points_gained.isna(), 'net_points_gained'] = 0


2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [4]:
pbp_data.head(5)

,game_id,play_id,home_team,away_team,season_type,posteam,defteam,touchdown,play_type,passer_player_id,...,incomplete_pass,interception,sack,series_result,sack_player_id,sack_player_name,fumbled_1_player_id,fumbled_1_player_name,series_points,net_points_gained
0,2018_01_ATL_PHI,1.0,PHI,ATL,REG,None,None,NaN,None,None,...,NaN,NaN,NaN,First down,None,None,None,None,NaN,0.0
1,2018_01_ATL_PHI,37.0,PHI,ATL,REG,ATL,PHI,0.0,kickoff,None,...,0.0,0.0,0.0,First down,None,None,None,None,NaN,0.0
2,2018_01_ATL_PHI,52.0,PHI,ATL,REG,ATL,PHI,0.0,no_play,None,...,0.0,0.0,0.0,First down,None,None,None,None,NaN,0.0
3,2018_01_ATL_PHI,75.0,PHI,ATL,REG,ATL,PHI,0.0,pass,00-0026143,...,0.0,0.0,0.0,First down,None,None,None,None,NaN,0.0
4,2018_01_ATL_PHI,104.0,PHI,ATL,REG,ATL,PHI,0.0,run,None,...,0.0,0.0,0.0,First down,None,None,None,None,NaN,0.0


### Some Notes
Most of the football season is the regular season.

Let's filter out all POST season games so that we can capture stats outside of high pressure and variable games.

Manipulations:
1. season_type == 'REG'
2. Filter out passing, rushing, and receiving player ids with our rosters dataset. We will merge these positions.
    - We only want QBs, RBs, WRs, and TEs.

In [17]:
who_cares_the_others = ['QB','RB','WR','TE']

In [18]:
# 1.
reg_season = pbp_data.loc[pbp_data.loc[:, 'season_type'] == 'REG']

In [19]:
# 2.
rosters = pd.read_csv('../Data/2018-2023_rosters.csv')[['season',
                                                        'position',
                                                        'player_id']]

# Passing
passing_df = reg_season.merge(rosters.rename(columns={'player_id': 'passer_player_id',
                                                            'position': 'passing_position'}), on=['season', 'passer_player_id'], how='left')

passing_df = passing_df.loc[passing_df.passing_position.isin(who_cares_the_others), :]


# Receiving
receiving_df = reg_season.merge(rosters.rename(columns={'player_id': 'receiver_player_id',
                                                             'position': 'receiving_position'}), on=['season', 'receiver_player_id'], how='left')

receiving_df = receiving_df.loc[receiving_df.receiving_position.isin(who_cares_the_others), :]

# Rushing
rushing_df = reg_season.merge(rosters.rename(columns={'player_id': 'rusher_player_id',
                                                      'position': 'rusher_position'}), on=['season', 'rusher_player_id'], how='left')

rushing_df = rushing_df.loc[rushing_df.rusher_position.isin(who_cares_the_others), :]

In [20]:
passing_df.columns

Index(['game_id', 'play_id', 'home_team', 'away_team', 'season_type',
       'posteam', 'defteam', 'touchdown', 'play_type', 'passer_player_id',
       'passer_player_name', 'receiver_player_id', 'receiver_player_name',
       'rusher_player_id', 'rusher_player_name', 'season', 'home_coach',
       'away_coach', 'yards_gained', 'ydsnet', 'incomplete_pass',
       'interception', 'sack', 'series_result', 'sack_player_id',
       'sack_player_name', 'fumbled_1_player_id', 'fumbled_1_player_name',
       'series_points', 'net_points_gained', 'passing_position'],
      dtype='object')

### Who's Doing What

In [21]:
# Passing
category_counts = passing_df.passing_position.value_counts()

categories = category_counts.index
counts = category_counts.values

fig = go.Figure(data=[go.Bar(x=categories, y=counts)])

fig.update_layout(
    title='Passing Plays by Position',
    xaxis_title='Position',
    yaxis_title='Count'
)

display(fig.show())

# Receiving
category_counts = receiving_df.receiving_position.value_counts()

categories = category_counts.index
counts = category_counts.values

fig = go.Figure(data=[go.Bar(x=categories, y=counts)])

fig.update_layout(
    title='Receiving Plays by Position',
    xaxis_title='Position',
    yaxis_title='Count'
)

display(fig.show())

# Rushing
category_counts = rushing_df.rusher_position.value_counts()

categories = category_counts.index
counts = category_counts.values

fig = go.Figure(data=[go.Bar(x=categories, y=counts)])

fig.update_layout(
    title='Rushing Plays by Position',
    xaxis_title='Position',
    yaxis_title='Count'
)

# display(fig.show())



None

None

### Let's Focus On Rushing!

Rushing is very easy. There is no change in possession therefore all yards are attribute to these beasts!

Features Considering:
   - game_id
   - play_id
   - touchdown
   - rusher_player_id
   - rusher_player_name
   - season
   - yards_gained
   - yards_gained
   - net_points_gained

In [22]:
rushing_df_necessary_cols = rushing_df.loc[:, ['game_id','play_id','rusher_player_id','rusher_player_name', 'touchdown',
                                               'season','yards_gained','series_result','net_points_gained']]

In [23]:
rushing_game_agg = rushing_df_necessary_cols.groupby(by=['rusher_player_id','rusher_player_name','game_id','season']).agg({'yards_gained': 'sum',
                                                                                                      'net_points_gained': 'sum',
                                                                                                      'touchdown': 'sum'}).reset_index().rename(columns={'yards_gained': 'rushings_yards_gained',
                                                                                                                                                         'net_points_gained': 'rushings_net_points_gained',
                                                                                                                                                         'touchdown': 'rushings_touchdown'})

### Now Passing

In [24]:
passing_df_necessary_cols = passing_df.loc[:, ['game_id','play_id','passer_player_id','passer_player_name', 'touchdown',
                                               'season','yards_gained','series_result','net_points_gained']]

In [25]:
passing_game_agg = passing_df_necessary_cols.groupby(by=['passer_player_id','passer_player_name','game_id','season']).agg({'yards_gained': 'sum',
                                                                                                      'net_points_gained': 'sum',
                                                                                                      'touchdown': 'sum'}).reset_index().rename(columns={'yards_gained': 'passing_yards_gained',
                                                                                                                                                         'net_points_gained': 'passing_net_points_gained',
                                                                                                                                                         'touchdown': 'passing_touchdown'})

### Now Receiving

In [26]:
receiving_necessary_cols = receiving_df.loc[:, ['game_id','play_id','receiver_player_id','receiver_player_name', 'touchdown',
                                               'season','yards_gained','series_result','net_points_gained']]

In [27]:
receiving_game_agg = receiving_necessary_cols.groupby(by=['receiver_player_id','receiver_player_name','game_id','season']).agg({'yards_gained': 'sum',
                                                                                                      'net_points_gained': 'sum',
                                                                                                      'touchdown': 'sum'}).reset_index().rename(columns={'yards_gained': 'receiving_yards_gained',
                                                                                                                                                         'net_points_gained': 'receiving_net_points_gained',
                                                                                                                                                         'touchdown': 'receiving_touchdown'})

### Combine them all

In [28]:
merge_one = rushing_game_agg.rename(columns={'rusher_player_id':'player_id', 'rusher_player_name':'player_name'}).merge(passing_game_agg.rename(columns={'passer_player_id':'player_id', 'passer_player_name':'player_name'}), on=['player_id', 'player_name', 'game_id','season'], how='outer')

In [29]:
merge_two = merge_one.merge(receiving_game_agg.rename(columns={'receiver_player_id':'player_id', 'receiver_player_name':'player_name'}), on=['player_id','player_name','game_id','season'], how='outer').fillna(0)

### Add in FFP points

In [30]:
bring_positions = merge_two.merge(rosters, on=['player_id', 'season'], how='inner')

In [31]:
filter_positions = bring_positions.loc[bring_positions.loc[:, 'position'].isin(who_cares_the_others), :]

In [32]:
filter_positions.loc[:, 'ffp'] = (filter_positions['rushings_yards_gained']/10) + \
                                 (filter_positions['passing_yards_gained']/25) + \
                                 (filter_positions['receiving_yards_gained']/10) + \
                                 (filter_positions['rushings_touchdown']*6) + \
                                 (filter_positions['passing_touchdown']*6) + \
                                 (filter_positions['receiving_touchdown']*6)

filter_positions = filter_positions.drop(columns=['rushings_yards_gained',
                                                  'passing_yards_gained',
                                                  'receiving_yards_gained',
                                                  'rushings_touchdown',
                                                  'passing_touchdown',
                                                  'receiving_touchdown'])

In [33]:
total_points_year = filter_positions.drop(columns=['game_id', 'player_id']).groupby(by=['player_name', 'position', 'season']).sum().reset_index()

In [34]:
weights = {
    2018: 0.5,
    2019: 0.6,
    2020: 0.7,
    2021: 0.8,
    2022: 0.9,
    2023: 1.0
}

In [35]:
total_points_year.loc[:, 'season_weight'] = total_points_year.loc[:, 'season'].map(weights)
total_points_year.loc[:, 'weighted_ffps'] = total_points_year.loc[:, 'ffp']*total_points_year.loc[:, 'season_weight']


In [36]:
average_player_agg = total_points_year.groupby(by=['player_name', 'player_id', 'position']).agg(weighted_ffps = ('weighted_ffps', 'mean'),
                                                                                                rushing_net_points_gained = ('rushings_net_points_gained', 'mean'),
                                                                                                passing_net_points_gained = ('passing_net_points_gained', 'mean'),
                                                                                                receiving_net_points_gained = ('receiving_net_points_gained', 'mean')).reset_index()


KeyError: 'player_id'

In [118]:
average_position_agg = average_player_agg.groupby(by=['position']).agg(weighted_ffps = ('weighted_ffps', 'mean')).reset_index().rename(columns={'weighted_ffps': 'weighted_ffps_position'})

In [119]:
normalize = average_player_agg.merge(average_position_agg, on=['position'], how='left')

In [120]:
normalize.loc[:, 'normalized_ffb_points'] = normalize.loc[:, 'weighted_ffps'] / normalize.loc[:, 'weighted_ffps_position']

In [128]:
import plotly.express as px
fig = px.bar(normalize[normalize['position'] == 'QB'].sort_values(by='normalized_ffb_points', ascending=False).head(10), x='player_name', y='normalized_ffb_points')
display(fig.show())


None

In [129]:
fig = px.bar(normalize[normalize['position'] == 'RB'].sort_values(by='normalized_ffb_points', ascending=False).head(10), x='player_name', y='normalized_ffb_points')
display(fig.show())

None

In [130]:
fig = px.bar(normalize[normalize['position'] == 'WR'].sort_values(by='normalized_ffb_points', ascending=False).head(10), x='player_name', y='normalized_ffb_points')
display(fig.show())

None

In [131]:
fig = px.bar(normalize[normalize['position'] == 'TE'].sort_values(by='normalized_ffb_points', ascending=False).head(10), x='player_name', y='normalized_ffb_points')
display(fig.show())

None